In [1]:
# import the necessary libraries
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import pandas as pd
import imutils
import time
import dlib
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def eye_aspect_ratio(eye):
    # Vertical
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])  
    # Horizontal
    C = dist.euclidean(eye[0], eye[3])
    # EAR
    ear = (A + B)/(2.0 * C)
    
    return ear

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [4]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [5]:
thresh = pd.read_csv(r'C:\Users\pnanp\Desktop\Minor.AI4S.S6\Personal Project\threshf3p1.csv',index_col = 0)

In [6]:
# ATTRIBUTE DERIVATION

data = pd.DataFrame()
dir = r'C:\Users\pnanp\Desktop\Minor.AI4S.S6\Personal Project\Data\Fold3_part1'

# Looping through directory for mp4 files to read
for file in os.listdir(dir):
    if file.endswith(".mp4"):
        path=os.path.join(dir, file)
        print('Initiating {}'.format(file))
        
        # Setting thresholds
        avgEarThresh = thresh.loc[thresh.index[thresh.State == file]].iloc[0,0]
        eyeThresh = (avgEarThresh*90)/100
        consecFrameThresh = 3
        
        # Setting counters
        blinkCount = 0
        blinkFrameCounter = 0
        frameCount = 0
        plinkFrameCounter = 0
        eyeCloseFrame = 0
        
        # Defining lists to store attributes
        allEars = []
        Duration = []
        Amplitude = []
        Frequency = []
        Frame = []
        PERCLOS = []
        
        # Read the video
        vs = FileVideoStream(path)
        vs.start()
        time.sleep(1.0)
        
        # While frame is incoming, read the frame
        while vs.more() == True:  
            frame = vs.read()
            # If the frame is not of type None, perform adjustments and face detection
            if vs.more() == True:
                frame =  imutils.resize(frame, width=450)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frameCount += 1     # Count and track each frame

                rects = detector(gray, 0)
                
                for rect in rects:
                    # Predict the landmarks and convert their coordinates into an array
                    shape =  predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    # Extract the coordinates of left and right eyesusing the indexes derived before
                    leftEye = shape[lStart:lEnd]
                    rightEye = shape[rStart:rEnd]
                    # Calculate EARs and take average
                    leftEAR = eye_aspect_ratio(leftEye)
                    rightEAR = eye_aspect_ratio(rightEye)

                    ear = (leftEAR + rightEAR) / 2.0

                    allEars.append(ear)     # List to track all EARs
                    
                    # Check threshold and current EAR
                    if ear < eyeThresh:
                        plinkFrameCounter += 1     # potential blink frame counter
                        plinkEars.append(ear)      # List containing all EAR of closed eyes duration
                    # The instance that EAR comes above the threshold   
                    else:    
                        # Check duration potential blink against frame threshold
                        if plinkFrameCounter >= consecFrameThresh:
                            blinkCount += 1
                            eyeCloseFrame = eyeCloseFrame + plinkFrameCounter

                            bottomEar = min(plinkEars)    # Lowest EAR is the min from the tracked list

                            if max(allEars[-6:]) > eyeThresh:
                                startEar = max(allEars[-6:])
                            else:
                                j = 7
                                while max(allEars[-j:]) <= eyeThresh:
                                    j += 1
                                startEar = max(allEars[-(j+3):])

                            duration = plinkFrameCounter
                            Duration.append(duration)

                            frequency = blinkCount/frameCount
                            Frequency.append(frequency)
                            Frame.append(frameCount)

                            amplitude = startEar - bottomEar
                            Amplitude.append(amplitude)
                            
                            perclos = 100*eyeCloseFrame/frameCount
                            PERCLOS.append(perclos)
                            
                        # Resetting potential blink counter and EAR list
                        plinkFrameCounter = 0
                        plinkEars = []
                    
        print('{} features extracted'.format(file))
        vs.stop()
        cv2.destroyAllWindows()
        
        # Creating and extending dataframe
        mediumDf = pd.DataFrame({'Duration': Duration, 'Amplitude': Amplitude, 
                                 'Frequency': Frequency, 'State': file, 'PERCLOS':PERCLOS})
        data = data.append(mediumDf, ignore_index = True)
        print('Data updated to {} rows'.format(len(data)))
        print('----------------------------------------')

Initiating 0s26.mp4
0s26.mp4 features extracted
Data updated to 40 rows
----------------------------------------
Initiating 0s27.mp4
0s27.mp4 features extracted
Data updated to 104 rows
----------------------------------------
Initiating 0s28.mp4
0s28.mp4 features extracted
Data updated to 171 rows
----------------------------------------
Initiating 0s29.mp4
0s29.mp4 features extracted
Data updated to 234 rows
----------------------------------------
Initiating 0s30.mp4
0s30.mp4 features extracted
Data updated to 309 rows
----------------------------------------
Initiating 10s26.mp4
10s26.mp4 features extracted
Data updated to 373 rows
----------------------------------------
Initiating 10s27.mp4
10s27.mp4 features extracted
Data updated to 467 rows
----------------------------------------
Initiating 10s28.mp4
10s28.mp4 features extracted
Data updated to 516 rows
----------------------------------------
Initiating 10s29.mp4
10s29.mp4 features extracted
Data updated to 623 rows
--------

In [7]:
data.head() # Top rows

,Duration,Amplitude,Frequency,State,PERCLOS
0,4,0.080071,0.006135,0s26.mp4,2.453988
1,3,0.049686,0.002899,0s26.mp4,1.014493
2,3,0.094133,0.004144,0s26.mp4,1.381215
3,18,0.171738,0.004310,0s26.mp4,3.017241
4,3,0.045268,0.005365,0s26.mp4,3.326180


In [8]:
data.to_csv('f3p1.csv')